# InfoGAN Tutorial

This tutorials walks through an implementation of InfoGAN as described in [InfoGAN: Interpretable Representation Learning by Information Maximizing Generative Adversarial Nets](https://arxiv.org/abs/1606.03657).

To learn more about InfoGAN, see this [Medium post](https://medium.com/p/dd710852db46) on them. To lean more about GANs generally, see [this one](https://medium.com/@awjuliani/generative-adversarial-networks-explained-with-a-classic-spongebob-squarepants-episode-54deab2fce39#.692jyamki).

In [1]:
#Import the libraries we will need.
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import os
import scipy.misc
import scipy

Load the MNIST dataset.

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Helper Functions

In [3]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

## Defining the Adversarial Networks

### Generator Network

The generator takes a vector of random numbers and transforms it into a 32x32 image. Each layer in the network involves a strided  transpose convolution, batch normalization, and rectified nonlinearity. Tensorflow's slim library allows us to easily define each of these layers.

In [4]:
# https://www.tensorflow.org/api_docs/python/tf/depth_to_space
# http://qiita.com/tadOne/items/48302a399dcad44c69c8   Tensorflow - padding = VALID/SAMEの違いについて
#     so 3 tf.depth_to_space(genX,2) gives 4x2^3 = 32
# 

def generator(z):
    
    zP = slim.fully_connected(z,4*4*256,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,256])
    
    gen1 = slim.convolution2d(\
        zCon,num_outputs=128,kernel_size=[3,3],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    gen1 = tf.depth_to_space(gen1,2)
    
    gen2 = slim.convolution2d(\
        gen1,num_outputs=64,kernel_size=[3,3],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    gen2 = tf.depth_to_space(gen2,2)
    
    gen3 = slim.convolution2d(\
        gen2,num_outputs=32,kernel_size=[3,3],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    gen3 = tf.depth_to_space(gen3,2)
    
    g_out = slim.convolution2d(\
        gen3,num_outputs=1,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network
The discriminator network takes as input a 32x32 image and transforms it into a single valued probability of being generated from real-world data. Again we use tf.slim to define the convolutional layers, batch normalization, and weight initialization.

In [5]:
def discriminator(bottom, cat_list,conts, reuse=False):
    
    dis1 = slim.convolution2d(bottom,32,[3,3],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    dis1 = tf.space_to_depth(dis1,2)
    
    dis2 = slim.convolution2d(dis1,64,[3,3],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    dis2 = tf.space_to_depth(dis2,2)
    
    dis3 = slim.convolution2d(dis2,128,[3,3],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    dis3 = tf.space_to_depth(dis3,2)
        
    dis4 = slim.fully_connected(slim.flatten(dis3),1024,activation_fn=lrelu,\
        reuse=reuse,scope='d_fc1', weights_initializer=initializer)
        
    d_out = slim.fully_connected(dis4,1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    q_a = slim.fully_connected(dis4,128,normalizer_fn=slim.batch_norm,\
        reuse=reuse,scope='q_fc1', weights_initializer=initializer)
    
    
    ## Here we define the unique layers used for the q-network. The number of outputs depends on the number of 
    ## latent variables we choose to define.
    q_cat_outs = []
    for idx,var in enumerate(cat_list):
        q_outA = slim.fully_connected(q_a,var,activation_fn=tf.nn.softmax,\
            reuse=reuse,scope='q_out_cat_'+str(idx), weights_initializer=initializer)
        q_cat_outs.append(q_outA)
    
    q_cont_outs = None
    if conts > 0:
        q_cont_outs = slim.fully_connected(q_a,conts,activation_fn=tf.nn.tanh,\
            reuse=reuse,scope='q_out_cont_'+str(conts), weights_initializer=initializer)
    
    return d_out,q_cat_outs,q_cont_outs

### Connecting them together

In [6]:
# https://www.tensorflow.org/api_docs/python/tf/split
# https://www.tensorflow.org/api_docs/python/tf/one_hot
# https://www.tensorflow.org/api_docs/python/tf/concat
# https://www.tensorflow.org/api_docs/python/tf/reduce_sum
# https://www.tensorflow.org/api_docs/python/tf/reduce_mean
# https://www.tensorflow.org/api_docs/python/tf/trainable_variables
# https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
# https://deepage.net/deep_learning/2016/10/26/batch_normalization.html
# z_lat: one_hot_size + z_size + number_continuous = 10+64+2=76
# g_loss def is interesting, my understanding: 
#        if Dg is the probablity to be told as feak data, then 1-Dg is the probabily of suceessfully cheating, 
#        so we cal KL(Dg/(1-Dg)), and readuce_mean works as sampling proceduce
# 

tf.reset_default_graph()

z_size = 64 #Size of initial z vector used for generator.

# Define latent variables.
# categorical_list = [10] # Each entry in this list defines a categorical variable of a specific size.
categorical_list = [10,10] # Each entry in this list defines a categorical variable of a specific size.
number_continuous = 2 # The number of continous variables.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,1],dtype=tf.float32) #Real images

#These placeholders load the latent variables.
latent_cat_in = tf.placeholder(shape=[None,len(categorical_list)],dtype=tf.int32)
#print("latent_cat_in:", latent_cat_in)
latent_cat_list = tf.split(latent_cat_in,len(categorical_list),1)
#print("latent_cat_list: ",latent_cat_list)
latent_cont_in = tf.placeholder(shape=[None,number_continuous],dtype=tf.float32)

oh_list = []
for idx,var in enumerate(categorical_list):
    latent_oh = tf.one_hot(tf.reshape(latent_cat_list[idx],[-1]),var)
    #print(latent_cat_list[idx])
    #print(latent_oh),  woundn't print anything in sess.run()
    oh_list.append(latent_oh)

#Concatenate all c and z variables.
z_lats = oh_list[:]
#print("1st z_lats: ", z_lats )
z_lats.append(z_in)
#print("2nd z_lats: ", z_lats )
z_lats.append(latent_cont_in)
#print("3rd z_lats: ", z_lats )
z_lat = tf.concat(z_lats,1)
#print("z_lat: ", z_lat )

Gz = generator(z_lat) #Generates images from random z vectors
Dx,_,_ = discriminator(real_in,categorical_list,number_continuous) #Produces probabilities for real images
Dg,QgCat,QgCont = discriminator(Gz,categorical_list,number_continuous,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
g_loss = -tf.reduce_mean(tf.log((Dg/(1-Dg)))) #KL Divergence optimizer

#Combine losses for each of the categorical variables.
cat_losses = []
for idx,latent_var in enumerate(oh_list):
    #print ("latent_var: ", latent_var)
    #print ("tf.log(QgCat[idx]): ",tf.log(QgCat[idx]))
    cat_loss = -tf.reduce_sum(latent_var*tf.log(QgCat[idx]),axis=1)
    cat_losses.append(cat_loss)
    
#Combine losses for each of the continous variables.
if number_continuous > 0:
    q_cont_loss = tf.reduce_sum(0.5 * tf.square(latent_cont_in - QgCont),axis=1)
else:
    q_cont_loss = tf.constant(0.0)

q_cont_loss = tf.reduce_mean(q_cont_loss)
q_cat_loss = tf.reduce_mean(cat_losses)
q_loss = tf.add(q_cat_loss,q_cont_loss)
tvars = tf.trainable_variables()
#print (len(tvars))
#for i in tvars:
#    print(i)

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.002,beta1=0.5)
trainerQ = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
d_grads = trainerD.compute_gradients(d_loss,tvars[9:-2-((number_continuous>0)*2)-(len(categorical_list)*2)]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss, tvars[0:9]) #Only update the weights for the generator network.
q_grads = trainerQ.compute_gradients(q_loss, tvars) 

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)
update_Q = trainerQ.apply_gradients(q_grads)

## Training the network
Now that we have fully defined our network, it is time to train it!

In [11]:
# on at52 (GTX1080), 15mins/10000 epochs , 5000000 is about 12.5 hrs　 
# https://stackoverflow.com/questions/19349410/how-to-pad-with-zeros-a-tensor-along-some-axis-python
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html
# blow up after 81800
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html
# https://www.tensorflow.org/api_docs/python/tf/Session#run
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html
c_val = 10
batch_size = 64 #Size of image batch to apply at each iteration.
#iterations = 500000 #Total number of iterations to use.
iterations = 81000 #Total number of iterations to use.
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to save trained model to.

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    for i in range(iterations):
        zs = np.random.uniform(-1.0,1.0,size=[batch_size,z_size]).astype(np.float32) #Generate a random z batch
        #print("zs shape:",zs.shape)
        #lcat = np.random.randint(0,10,[batch_size,len(categorical_list)]) #Generate random c batch
        lcat = np.random.randint(0,c_val,[batch_size,len(categorical_list)]) #Generate random c batch
        #print("lcat", lcat)
        #print("lcat shape: ", lcat.shape)
        lcont = np.random.uniform(-1,1,[batch_size,number_continuous]) #
        
        xs,_ = mnist.train.next_batch(batch_size) #Draw a sample batch from MNIST dataset.
        xs = (np.reshape(xs,[batch_size,28,28,1]) - 0.5) * 2.0 #Transform it to be between -1 and 1
        xs = np.lib.pad(xs, ((0,0),(2,2),(2,2),(0,0)),'constant', constant_values=(-1, -1)) #Pad the images so the are 32x32
        
        _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the discriminator
        _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the generator, twice for good measure.
        _,qLoss,qK,qC = sess.run([update_Q,q_loss,q_cont_loss,q_cat_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update to optimize mutual information.
        if i % 100 == 0:
            print ("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
            z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
            #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
            lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
            latent_fixed = np.ones((c_val*c_val,1))
            lcat_sample = np.hstack([latent_fixed,lcat_sample])
            
            #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
            a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
            #b = np.reshape(a,[100,1])
            b = np.reshape(a,[c_val*c_val,1])
            c = np.zeros_like(b)
            lcont_sample = np.hstack([b,c])
            #
            samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig'+str(i)+'.png')
            save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig'+str(i)+'.png')
        if i % 1000 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print ("Saved Model on ", i)

Gen Loss: 2.95689 Disc Loss: 1.35589 Q Losses: [0.38443068, 2.3201294]
Gen Loss: 4.57285 Disc Loss: 0.401731 Q Losses: [0.28363293, 2.2566705]
Gen Loss: 0.333817 Disc Loss: 1.66211 Q Losses: [0.30599505, 2.3136113]
Gen Loss: 3.24278 Disc Loss: 0.441559 Q Losses: [0.27389646, 2.2756143]
Gen Loss: 3.78939 Disc Loss: 0.284224 Q Losses: [0.2178901, 2.2373662]
Gen Loss: 2.28797 Disc Loss: 0.612261 Q Losses: [0.19002303, 2.2524757]
Gen Loss: 0.384185 Disc Loss: 0.85043 Q Losses: [0.17324576, 2.2355111]
Gen Loss: 6.0405 Disc Loss: 3.06843 Q Losses: [0.13084136, 2.1701374]
Gen Loss: 1.65668 Disc Loss: 1.25725 Q Losses: [0.16160688, 2.0884387]
Gen Loss: 0.693242 Disc Loss: 0.835535 Q Losses: [0.13886118, 1.8785381]
Gen Loss: 1.94354 Disc Loss: 0.815342 Q Losses: [0.12573269, 1.8651717]
Saved Model on  1000
Gen Loss: 1.81809 Disc Loss: 0.601826 Q Losses: [0.11081026, 1.6222768]
Gen Loss: -0.886611 Disc Loss: 1.36294 Q Losses: [0.11593219, 1.6118336]
Gen Loss: 3.21888 Disc Loss: 0.7391 Q Losses: 

## Using a trained network
Once we have a trained model saved, we may want to use it to generate new images, and explore the representation it has learned.

In [18]:
# http://qiita.com/TokyoMickey/items/f6a9251f5a59120e39f8

sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to load trained model from.

#init = tf.initialize_all_variables()

saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    #sess.run(init)
    #Reload the model.
    print ('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
    lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
    #lcat_sample = np.reshape(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]),[100,1])
    #print(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]))
    a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
    b = np.reshape(a,[100,1])
    c = np.zeros_like(b)
    #c = np.zeros_like(b)+8
    lcont_sample = np.hstack([b,c])
    samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
    if not os.path.exists(sample_directory):
        os.makedirs(sample_directory)
    #Save sample generator images for viewing training progress.
    save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test'+'.png')
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test_4'+'.png')

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/model-80000.cptk
